### Install the libraries

In [1]:
!pip install pinecone llama-index-vector-stores-pinecone -q


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


### Import the libraries

In [9]:
from llama_index.core import(
    Settings, 
    VectorStoreIndex, 
    StorageContext,
    SimpleDirectoryReader
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os
import time

### Initialize the APIs

In [ ]:
# loading the .env file
load_dotenv()

# initializing Groq and Pinecone APIs
GROQ_API_KEY = os.environ["GROQ_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key = PINECONE_API_KEY)

### LLM and Embedding Model Configuration

In [4]:
llm = Groq(model = "llama-3.1-8b-instant")
embed_model = HuggingFaceEmbedding(model_name = "sentence-transformers/all-mpnet-base-v2")

Settings.llm = llm
Settings.embed_model = embed_model

README.md: 0.00B [00:00, ?B/s]

### Pinecone Vector DB Creation

In [ ]:
# setting index_name
index_name = "llama3-groq-pinecone"

# checking existing indexes
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# creating an index if it doesn't exists
if index_name not in existing_indexes: 
    pc.create_index(
        name = index_name, 
        dimension = 768, 
        metric = "cosine", 
        spec = ServerlessSpec(
                cloud = "aws", 
                region = "us-east-1"
        )
    )

# connecting to the created index
index = pc.Index(index_name)
time.sleep(1)

# describing index statistics
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

### Loading the data

In [7]:
documents = documents = SimpleDirectoryReader("sample_data").load_data()

### Using Pinecone Vector Store

In [10]:
vector_store = PineconeVectorStore(pinecone_index = index)
storage_context = StorageContext.from_defaults(vector_store = vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context = storage_context
)

Upserted vectors:   0%|          | 0/12 [00:00<?, ?it/s]

### Querying the stored data

In [15]:
query_engine = index.as_query_engine()
response = query_engine.query("What are the different challenges in Copilots?")

2025-10-13 21:15:35,736 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [16]:
print(response)

There are several challenges associated with Copilots, including:

1. Ensuring accuracy and reliability: Copilots must be able to provide accurate and reliable information, which can be a challenge, especially when dealing with complex or nuanced topics.
2. Adapting to user preferences: Copilots need to be able to adapt to individual user preferences and needs, which can be a challenge, especially if users have different communication styles or preferences.
3. Handling ambiguity and uncertainty: Copilots often encounter ambiguous or uncertain language, which can be challenging to interpret and respond to.
4. Maintaining knowledge base relevance: Copilots rely on a knowledge base that must be regularly updated and maintained to ensure it remains relevant and accurate.
5. Balancing creativity and accuracy: Copilots must balance creativity and accuracy, as they need to provide innovative solutions while also ensuring that they are accurate and reliable.
6. Managing user expectations: Copi